<a href="https://colab.research.google.com/github/ViniciusCantanhede/DesafioDS-NLP/blob/main/DesafioNeuralMed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import unicodedata
import nltk
from nltk.tokenize import word_tokenize
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from transformers import DataCollatorForTokenClassification
from datasets import Dataset, DatasetDict
import json

In [ ]:
from google.colab import userdata
import os
from huggingface_hub import login

token = userdata.get('HUGGINGFACE_TOKEN')
login(token=token)

In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Verifica se GPU está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilizando: {device}")

Utilizando: cuda


# Carregando dados

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Desafio/sample_teste_nlp.csv")
print(f"Dataset com {len(df)} documentos")
df.head()

Dataset com 6834 documentos


,patient_id,age,sex,ehr_date,text
0,341196,82.0,M,2023-09-05 10:23:37,"<html tasy=""html5""><body><p style=""text-align:..."
1,4309949,32.0,F,2023-11-01 12:10:59,"<html tasy=""html5""><body><p style=""text-align:..."
2,404932,91.0,M,2023-09-16 07:24:02,{\rtf1\ansi\ansicpg1252{\fonttbl{\f0\fnil\fcha...
3,5603035,53.0,F,2024-08-03 06:56:48,RET 6 MESES PARA AVALIAÇÃO CALCIO + VIT D
4,154876,27.0,M,2023-11-05 04:58:01,{\rtf1\ansi\ansicpg1252{\fonttbl{\f0\fnil\fcha...


#Funções de Pré-processamento
#### `Remove_rtf_tags:`
- Remove formatação RTF de documentos usando expressões regulares para limpar tags como \fonttbl, \colortbl, etc.
- O RTF (Rich Text Format) é um formato de arquivo desenvolvido pela Microsoft que permite a criação e a troca de documentos com formatação entre diferentes plataformas e aplicativos.

In [ ]:
def remove_rtf_tags(text):
  #removendo tags padrão
  rtf_pattern = r'\\[a-z]+[0-9]?'
  #Removendo tags específicas
  clean_text = re.sub(rtf_pattern, '', text)
  #Removendo caracters de controle
  clean_text = re.sub(r'[{}]', '', clean_text)
  # elementos adicionais
  clean_text = re.sub(r'\\[a-z]+[0-9]?[a-z]*', '', clean_text)
  return clean_text

#### `Remove_html_tags:`Limpa tags HTML de documentos, substituindo elementos por espaços e removendo formatação.

In [ ]:
def remove_html_tags(text):
    #padrão para extrair texto de tags HTML
    clean_text = re.sub(r'<.*?>', ' ', text)
    # Removendo multiplos espaços
    clean_text = re.sub(r'\s+', ' ', clean_text)
    return clean_text.strip()

#### `Normalize_text`: Normaliza o texto removendo acentos e convertendo para minúsculas

In [ ]:
def normalize_text(text):
    #Normalização Unicode NFKD
    text = unicodedata.normalize('NFKD', text) # especialmente importante para línguas com acentos como o português.
    #Remove diacríticos
    text = ''.join([c for c in text if not unicodedata.combining(c)])
    return text.lower()

#### `Extract_timestamps:` Identifica padrões de horário (ex: "07:00HS") e segmenta o texto cronologicamente, organizando em pares de timestamp e conteúdo.

In [ ]:
def extract_timestamps(text):
    #Padrão para encontrar horários
    timestamp_pattern = r'(\d{1,2}:\d{2})(?:HS|H)'
    # Encontra todos os timestamps
    timestamps = re.findall(timestamp_pattern, text)
    #divide o texto pelos timestamps
    segments = re.split(timestamp_pattern, text)
    # Remove o primeiro segmento vazio se houver
    segments = [seg.strip() for seg in segments if seg.strip()]
    # Organiza os segmentos com seus timestamps
    time_segments = []
    for i in range(0, len(segments), 2):
        if i+1 < len(segments):
            time_segments.append({
                'timestamp': segments[i],
                'text': segments[i+1]
            })
    return time_segments

#### `Tokenize_medical_text`: Implementa tokenização especializada para textos médicos, tratando adequadamente abreviações como "mg", "ml" e unindo valores numéricos com suas unidades.

In [ ]:
def tokenize_medical_text(text):
    # lista de abreviações médicas comuns em português
    medical_abbr = ['mg', 'ml', 'g', 'mcg', 'ev', 'im', 'vo', 'sc', 'sl', 'cp', 'amp', 'fr']
    # tokenização básica
    tokens = word_tokenize(text)
    #trata tokens com abreviações médicas
    processed_tokens = []
    i = 0
    while i < len(tokens):
        if i+1 < len(tokens) and tokens[i].isdigit() and tokens[i+1] in medical_abbr:
            # Une valores numéricos com suas unidades
            processed_tokens.append(tokens[i] + tokens[i+1])
            i += 2
        else:
            processed_tokens.append(tokens[i])
            i += 1
    return processed_tokens

#### `Clean_medical_text:` Função unificada que detecta automaticamente o formato do texto (HTML, RTF ou texto puro) e aplica a limpeza apropriada.

In [ ]:
def clean_medical_text(text):
    # Verifica o formato e aplica a limpeza apropriada
    if '<html' in text:
        cleaned_text = remove_html_tags(text)
    elif '{\\rtf' in text:
        cleaned_text = remove_rtf_tags(text)
    else:
        cleaned_text = text
    # Normaliza o texto limpo
    normalized_text = normalize_text(cleaned_text)
    return normalized_text

# Dicionários de Entidades Médicas

#### Dicionários apra reconhecimento de entidades médicas
- `MEDICAMENTOS:` Lista de medicamentos comuns
- `PROCEDIMENTOS:` Procedimentos e intervenções médicas
- `DISPOSITIVOS:` Equipamentos e materiais médicos
- `CONDICOES:` Estados e condições do paciente

In [ ]:
MEDICAMENTOS = {
    'insulina': 'MEDICAMENTO',
    'regular': 'MEDICAMENTO',
    'gliconato': 'MEDICAMENTO',
    'calcio': 'MEDICAMENTO',
    'sf': 'MEDICAMENTO',
    'soro': 'MEDICAMENTO',
    'fisiologico': 'MEDICAMENTO',
    'sg': 'MEDICAMENTO',
    'glicosado': 'MEDICAMENTO',
    'polarizante': 'MEDICAMENTO',
    'glicose': 'MEDICAMENTO',
    'dipirona': 'MEDICAMENTO',
    'metoclopramida': 'MEDICAMENTO',
    'ceftriaxona': 'MEDICAMENTO',
    'omeprazol': 'MEDICAMENTO',
    'furosemida': 'MEDICAMENTO',
    'paracetamol': 'MEDICAMENTO',
    'morfina': 'MEDICAMENTO',
    'losartana': 'MEDICAMENTO',
    'captopril': 'MEDICAMENTO',
    'midazolam': 'MEDICAMENTO',
    'fentanil': 'MEDICAMENTO',
    'noradrenalina': 'MEDICAMENTO',
    'heparina': 'MEDICAMENTO',
    'cetoprofeno': 'MEDICAMENTO',
    'ondansetrona': 'MEDICAMENTO',
    'amoxicilina': 'MEDICAMENTO'
}

PROCEDIMENTOS = {
    'verificado': 'PROCEDIMENTO',
    'realizado': 'PROCEDIMENTO',
    'teste': 'PROCEDIMENTO',
    'glicemico': 'PROCEDIMENTO',
    'coleta': 'PROCEDIMENTO',
    'swab': 'PROCEDIMENTO',
    'medicada': 'PROCEDIMENTO',
    'troca': 'PROCEDIMENTO',
    'encaminhada': 'PROCEDIMENTO',
    'monitorizada': 'PROCEDIMENTO',
    'administrado': 'PROCEDIMENTO',
    'aferido': 'PROCEDIMENTO',
    'puncionado': 'PROCEDIMENTO',
    'aspirado': 'PROCEDIMENTO',
    'intubado': 'PROCEDIMENTO',
    'extubado': 'PROCEDIMENTO',
    'banho': 'PROCEDIMENTO',
    'higiene': 'PROCEDIMENTO',
    'curativo': 'PROCEDIMENTO',
    'avaliacao': 'PROCEDIMENTO',
    'exame': 'PROCEDIMENTO',
    'tomografia': 'PROCEDIMENTO',
    'raio': 'PROCEDIMENTO'
}

DISPOSITIVOS = {
    'cateter': 'DISPOSITIVO',
    'o2': 'DISPOSITIVO',
    'oxigenio': 'DISPOSITIVO',
    'avp': 'DISPOSITIVO',
    'acesso': 'DISPOSITIVO',
    'venoso': 'DISPOSITIVO',
    'mse': 'DISPOSITIVO',
    'salinizado': 'DISPOSITIVO',
    'pulseira': 'DISPOSITIVO',
    'sonda': 'DISPOSITIVO',
    'cvc': 'DISPOSITIVO',
    'dreno': 'DISPOSITIVO',
    'tubo': 'DISPOSITIVO',
    'mascara': 'DISPOSITIVO',
    'monitor': 'DISPOSITIVO',
    'bomba': 'DISPOSITIVO',
    'jelco': 'DISPOSITIVO',
    'oximetro': 'DISPOSITIVO',
    'ventilador': 'DISPOSITIVO',
    'respirador': 'DISPOSITIVO',
    'cpap': 'DISPOSITIVO'
}

CONDICOES = {
    'calma': 'CONDIÇÃO_PACIENTE',
    'sonolenta': 'CONDIÇÃO_PACIENTE',
    'acomodada': 'CONDIÇÃO_PACIENTE',
    'orientada': 'CONDIÇÃO_PACIENTE',
    'lucido': 'CONDIÇÃO_PACIENTE',
    'confuso': 'CONDIÇÃO_PACIENTE',
    'agitado': 'CONDIÇÃO_PACIENTE',
    'sedado': 'CONDIÇÃO_PACIENTE',
    'estavel': 'CONDIÇÃO_PACIENTE',
    'eupneico': 'CONDIÇÃO_PACIENTE',
    'dispneico': 'CONDIÇÃO_PACIENTE',
    'febril': 'CONDIÇÃO_PACIENTE',
    'afebril': 'CONDIÇÃO_PACIENTE',
    'hipotenso': 'CONDIÇÃO_PACIENTE',
    'hipertenso': 'CONDIÇÃO_PACIENTE',
    'normotenso': 'CONDIÇÃO_PACIENTE',
    'hidratado': 'CONDIÇÃO_PACIENTE',
    'desidratado': 'CONDIÇÃO_PACIENTE',
    'corado': 'CONDIÇÃO_PACIENTE',
    'descorado': 'CONDIÇÃO_PACIENTE',
    'icterico': 'CONDIÇÃO_PACIENTE',
    'cianotico': 'CONDIÇÃO_PACIENTE',
    'taquipneico': 'CONDIÇÃO_PACIENTE'
}

# Funções de Processamento e Anotação
#### `Process_document_batch`: Processa um lote de documentos do DataFrame, limpando e tokenizando cada um. Permite usar apenas uma amostra aleatória dos dados.

    Processa um lote de documentos do dataframe

    Args:
        df: DataFrame com os documentos médicos
        sample_size: Número de documentos a processar (opcional)

    Returns:
        Lista de documentos processados com tokens
    

In [ ]:
def process_document_batch(df, sample_size=None):
    if sample_size and sample_size < len(df):
        df_sample = df.sample(sample_size, random_state=42)
        print(f"Processando amostra de {sample_size} documentos")
    else:
        df_sample = df
        print(f"Processando todos os {len(df)} documentos")

    processed_documents = []
    for idx, row in df_sample.iterrows():
        if idx % 100 == 0:
            print(f"Processando documento {idx}/{len(df_sample)}")

        try:
            #extrai e limpa o texto
            text = row['text']
            cleaned_text = clean_medical_text(text)

            #pula documentos muito curtos com menos de 10 caracteres
            if len(cleaned_text) < 10:
                continue
            #Tokeniza o texto
            tokens = word_tokenize(cleaned_text)

            #Cria o documento processado
            processed_doc = {
                "patient_id": row["patient_id"],
                "tokens": tokens,
                "tags": ["O"] * len(tokens)  # Inicializa com tags "Outside"
            }
            processed_documents.append(processed_doc)
        except Exception as e:
            print(f"Erro ao processar documento {idx}: {str(e)}")

    print(f"Processamento concluído. Total de {len(processed_documents)} documentos processados.")
    return processed_documents

#### `Annotate_entities`: Anota entidades médicas nos documentos utilizando os dicionários e regras contextuais, atribuindo tags no formato BIO (Beginning-Inside-Outside).

- Nesse formato, cada token (ou palavra) recebe uma tag que indica se ele é o B (Beginning, início) de uma entidade, o I (Inside, dentro) da mesma entidade ou o O (Outside, fora) de qualquer entidade.
- Por exemplo, na frase "Maria Silva mora em São Paulo", se "Maria Silva" for considerada uma entidade do tipo pessoa, "Maria" receberia a tag B-PER e "Silva" a tag I-PER, enquanto "mora", "em", "São" e "Paulo" seriam anotados conforme o contexto, podendo ser O ou tags específicas se fizerem parte de outra entidade.

    
    Anota entidades médicas nos documentos processados

    Args:
        processed_documents: Lista de documentos processados com tokens

    Returns:
        Lista de documentos com entidades anotadas
    

In [ ]:
def annotate_entities(processed_documents):
    print("Anotando entidades médicas...")

    entity_counts = {
        "MEDICAMENTO": 0,
        "PROCEDIMENTO": 0,
        "DISPOSITIVO": 0,
        "CONDIÇÃO_PACIENTE": 0,
        "RESULTADO_EXAME": 0
    }

    annotated_documents = []
    for doc in processed_documents:
        tokens = doc["tokens"]
        tags = ["O"] * len(tokens)
        #Anota medicamentos
        for i, token in enumerate(tokens):
            if token in MEDICAMENTOS:
                tags[i] = "B-MEDICAMENTO"
                entity_counts["MEDICAMENTO"] += 1
                #Verifica tokens subsequentes
                j = i + 1
                while j < len(tokens) and tokens[j] in MEDICAMENTOS:
                    tags[j] = "I-MEDICAMENTO"
                    j += 1
        #anota procedimentos
        for i, token in enumerate(tokens):
            if token in PROCEDIMENTOS and tags[i] == "O":
                tags[i] = "B-PROCEDIMENTO"
                entity_counts["PROCEDIMENTO"] += 1
                j = i + 1
                while j < len(tokens) and tokens[j] in PROCEDIMENTOS and tags[j] == "O":
                    tags[j] = "I-PROCEDIMENTO"
                    j += 1
        # Anota dispositivos
        for i, token in enumerate(tokens):
            if token in DISPOSITIVOS and tags[i] == "O":
                tags[i] = "B-DISPOSITIVO"
                entity_counts["DISPOSITIVO"] += 1
                j = i + 1
                while j < len(tokens) and tokens[j] in DISPOSITIVOS and tags[j] == "O":
                    tags[j] = "I-DISPOSITIVO"
                    j += 1
        # Anota condições do paciente
        for i, token in enumerate(tokens):
            if token in CONDICOES and tags[i] == "O":
                tags[i] = "B-CONDIÇÃO_PACIENTE"
                entity_counts["CONDIÇÃO_PACIENTE"] += 1
                j = i + 1
                while j < len(tokens) and tokens[j] in CONDICOES and tags[j] == "O":
                    tags[j] = "I-CONDIÇÃO_PACIENTE"
                    j += 1
        # Anota resultados de exames (padrão numérico + unidade)
        for i, token in enumerate(tokens):
            if re.match(r'^\d+$', token) and i+1 < len(tokens):
                next_token = tokens[i+1].lower()
                if ('mg' in next_token or 'dl' in next_token or 'ml' in next_token):
                    tags[i] = "B-RESULTADO_EXAME"
                    tags[i+1] = "I-RESULTADO_EXAME"
                    entity_counts["RESULTADO_EXAME"] += 1
        # Adiciona documento anotado
        doc["tags"] = tags
        # Verifica se o documento tem pelo menos uma entidade
        if any(tag != "O" for tag in tags):
            annotated_documents.append(doc)

    # Exibe estatísticas de anotação
    total_entities = sum(entity_counts.values())
    print(f"\nTotal de documentos com entidades: {len(annotated_documents)}")
    print(f"Total de entidades anotadas: {total_entities}")

    print("\nDistribuição de entidades:")
    for entity_type, count in entity_counts.items():
        percentage = (count / total_entities * 100) if total_entities > 0 else 0
        print(f"- {entity_type}: {count} ({percentage:.1f}%)")

    return annotated_documents

# Desenvolvimento do modelo NER

#### `Prepare_dateset:`Prepara os exemplos anotados para o formato esperado pela biblioteca Hugging Face, dividindo em treino (80%) e validação (20%).

In [ ]:
#Define as etiquetas para NER
labels = ["O", "B-MEDICAMENTO", "I-MEDICAMENTO", "B-PROCEDIMENTO", "I-PROCEDIMENTO",
          "B-DISPOSITIVO", "I-DISPOSITIVO", "B-CONDIÇÃO_PACIENTE", "I-CONDIÇÃO_PACIENTE",
          "B-RESULTADO_EXAME", "I-RESULTADO_EXAME"]

# Mapeia as etiquetas para IDs
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for i, label in enumerate(labels)}

def prepare_dataset(examples):
    # Converte para o formato do Dataset do Hugging Face
    df = pd.DataFrame(examples)
    #Divide em treino e validação
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
    # converte para o formato de Dataset
    train_dataset = Dataset.from_pandas(train_df)
    val_dataset = Dataset.from_pandas(val_df)
    #cria o DatasetDict
    dataset_dict = DatasetDict({
        "train": train_dataset,
        "validation": val_dataset
    })
    return dataset_dict

#### `Tokenize_and_align_labels`:Alinha as etiquetas com os tokens gerados pelo tokenizer do modelo, tratando a segmentação de palavras.

In [ ]:
def tokenize_and_align_labels(examples, tokenizer):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding="max_length",
        max_length=128
    )

    labels = []
    for i, label in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None

        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label[word_idx]])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

#### `Train_ner_model:`Configura e treina o modelo NER usando transformers, salvando o resultado final.

In [ ]:
def train_ner_model(dataset_dict, model_name="neuralmind/bert-base-portuguese-cased"):
    # Inicializa o tokenizer e o modelo
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForTokenClassification.from_pretrained(
        model_name,
        num_labels=len(labels),
        id2label=id2label,
        label2id=label2id
    )
    #Tokeniza e alinha as etiquetas
    tokenized_datasets = dataset_dict.map(
        lambda examples: tokenize_and_align_labels(examples, tokenizer),
        batched=True
    )
    #Configura o data collator
    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
    # Configura o treinamento
    training_args = TrainingArguments(
        output_dir="./medical-ner-model",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
        save_strategy="epoch",
        load_best_model_at_end=True,
    )
    # Inicializa o trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        tokenizer=tokenizer,
        data_collator=data_collator,
    )
    # Treina o modelo
    trainer.train()
    # Salva o modelo e o tokenizer
    model.save_pretrained("./medical-ner-model-final")
    tokenizer.save_pretrained("./medical-ner-model-final")
    return model, tokenizer

# Funções de Predição e Avaliação

#### `Predict_entities`: Prediz entidades em um novo texto usando o modelo treinado, lidando corretamente com a transferência para GPU

In [ ]:
def predict_entities(text, model, tokenizer):
    # limpa e tokeniza o texto
    clean_text = clean_medical_text(text)
    tokens = word_tokenize(clean_text)

    #prepara as entradas para o modelo
    inputs = tokenizer(tokens, is_split_into_words=True, return_tensors="pt", padding=True, truncation=True)
    word_ids = inputs.word_ids(batch_index=0)
    # Move para GPU se disponível
    if torch.cuda.is_available():
        model = model.to('cuda')
        # Move apenas os tensores para GPU
        input_ids = inputs['input_ids'].to('cuda')
        attention_mask = inputs['attention_mask'].to('cuda')

        #Faz a predição
        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    else:
        #Faz a predição na CPU
        with torch.no_grad():
            outputs = model(**inputs)

    # Obtém as predições
    predictions = torch.argmax(outputs.logits, dim=2)
    # Mapeia as predições para tokens
    predicted_labels = []
    for token_idx, token_prediction in enumerate(predictions[0].cpu().numpy()):
        word_idx = word_ids[token_idx]
        if word_idx is not None:
            if len(predicted_labels) <= word_idx:
                predicted_labels.append(id2label[token_prediction])
    # Extrai as entidades
    entities = []
    current_entity = {"text": "", "label": "", "start": 0, "end": 0}
    for i, (token, label) in enumerate(zip(tokens, predicted_labels)):
        if label.startswith("B-"):
            if current_entity["text"]:
                entities.append(current_entity)
            entity_type = label[2:]  # Remove o prefixo B-
            current_entity = {
                "text": token,
                "label": entity_type,
                "start": i,
                "end": i + 1
            }
        elif label.startswith("I-") and current_entity["text"]:
            # Continua a entidade atual
            current_entity["text"] += " " + token
            current_entity["end"] = i + 1
        elif label == "O" and current_entity["text"]:
            # Termina a entidade atual
            entities.append(current_entity)
            current_entity = {"text": "", "label": "", "start": 0, "end": 0}
    # Adiciona a última entidade se existir
    if current_entity["text"]:
        entities.append(current_entity)
    return entities

#### `Evaluate_model_on_examples`: Avalia o modelo em exemplos predefinidos e apresenta estatísticas de performance.


    Avalia o modelo treinado em exemplos de teste

    Args:
        model: Modelo NER treinado
        tokenizer: Tokenizer do modelo
        examples: Lista de textos de exemplo para avaliar


In [ ]:
def evaluate_model_on_examples(model, tokenizer, examples):
    print("\nAvaliando o modelo em exemplos de teste:")

    for i, example in enumerate(examples):
        print(f"\nExemplo {i+1}: {example}")
        #Prediz entidades
        entities = predict_entities(example, model, tokenizer)
        #exibe entidades detectadas
        if entities:
            print("Entidades detectadas:")
            for entity in entities:
                print(f"- {entity['text']} ({entity['label']})")
        else:
            print("Nenhuma entidade detectada.")
    return True

#### `Visualize_entities`: Cria uma visualização das entidades detectadas no texto, marcando-as no formato texto.


    Visualiza as entidades detectadas em um texto

    Args:
        text: Texto original
        entities: Lista de entidades detectadas


In [ ]:
def visualize_entities(text, entities):
    #se não houver entidades retorna o texto original
    if not entities:
        return text
    #prepara o texto para visualização
    tokens = text.split()
    marked_text = list(tokens)
    # Ordena as entidades por posição (do fim para o início)
    entities = sorted(entities, key=lambda x: x["start"], reverse=True)
    #Substitui tokens pelas entidades marcadas
    for entity in entities:
        start = entity["start"]
        end = entity["end"]
        # Verifica se os índices são válidos
        if start < len(marked_text):
            # Obtém o texto da entidade
            entity_text = " ".join(marked_text[start:end])
            # Cria a marcação
            marked_entity = f"[{entity_text}]({entity['label']})"
            # Substitui os tokens
            marked_text[start] = marked_entity
            for i in range(start + 1, end):
                if i < len(marked_text):
                    marked_text[i] = ""
    # Remove tokens vazios e junta o texto
    marked_text = [t for t in marked_text if t]
    return " ".join(marked_text)

#### `train_medical_ner_pipeline:`Pipeline completo que integra todas as etapas, desde o processamento até a avaliação final.


    Pipeline completo para treinar um modelo NER para textos médicos

    Args:
        df: DataFrame com documentos médicos
        sample_size: Número de documentos para usar

    Returns:
        Modelo treinado e tokenizer


In [ ]:
def train_medical_ner_pipeline(df, sample_size=1000):
    print("\n" + "="*50)
    print("PIPELINE DE TREINAMENTO NER PARA TEXTOS MÉDICOS")
    print("="*50)

    # 1. Processamento de documentos
    print("\nETAPA 1: Processamento de documentos")
    processed_docs = process_document_batch(df, sample_size=sample_size)
    # 2. Anotação de entidades
    print("\nETAPA 2: Anotação de entidades")
    annotated_docs = annotate_entities(processed_docs)
    # 3. Preparação dos datasets
    print("\nETAPA 3: Preparação dos datasets")
    dataset_dict = prepare_dataset(annotated_docs)
    print(f"Dataset de treino: {len(dataset_dict['train'])} exemplos")
    print(f"Dataset de validação: {len(dataset_dict['validation'])} exemplos")
    # 4. Treinamento do modelo
    print("\nETAPA 4: Treinamento do modelo")
    model, tokenizer = train_ner_model(dataset_dict)
    # 5. Avaliação em exemplos
    print("\nETAPA 5: Avaliação do modelo")
    examples = [
        "Paciente apresenta-se calmo, consciente e orientado, com acesso venoso periférico em membro superior direito, recebendo ceftriaxona 1g EV de 12/12h.",
        "Realizado teste glicêmico com resultado de 145 mg/dl, administrado insulina regular 4 UI conforme prescrição.",
        "Paciente em uso de cateter de O2 a 2l/min, apresentando dispneia aos esforços e com monitorização contínua."
    ]

    evaluate_model_on_examples(model, tokenizer, examples)
    # 6. Salvamento do modelo
    print("\nETAPA 6: Salvamento do modelo")
    output_dir = "./medical-ner-model-final"
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"Modelo salvo em: {output_dir}")
    print("\n" + "="*50)
    print("PIPELINE DE TREINAMENTO CONCLUÍDO COM SUCESSO!")
    print("="*50)

    return model, tokenizer

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

#### `sample_size`: Número de documentos a serem usados para treinamento, 3000 se mostrou um número razoável, não demora muito para treinar e o seu desempenho é bom

In [ ]:
sample_size = 3000

# Treinamento do modelo
model, tokenizer = train_medical_ner_pipeline(df, sample_size=sample_size)


PIPELINE DE TREINAMENTO NER PARA TEXTOS MÉDICOS

ETAPA 1: Processamento de documentos
Processando amostra de 3000 documentos
Processando documento 1600/3000
Processando documento 800/3000
Processando documento 5100/3000
Processando documento 6000/3000
Processando documento 100/3000
Processando documento 4100/3000
Processando documento 5900/3000
Processando documento 4600/3000
Processando documento 1400/3000
Processando documento 3400/3000
Processando documento 6500/3000
Processando documento 6400/3000
Processando documento 4800/3000
Processando documento 4900/3000
Processando documento 5200/3000
Processando documento 3100/3000
Processando documento 2900/3000
Processando documento 1700/3000
Processando documento 1200/3000
Processando documento 300/3000
Processando documento 6300/3000
Processando documento 4700/3000
Processando documento 5700/3000
Processando documento 2600/3000
Processando documento 2500/3000
Processamento concluído. Total de 2991 documentos processados.

ETAPA 2: Anot

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

...

Map:   0%|          | 0/458 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-19-8190f3d4d8c7>:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,0.048627
2,No log,0.024122
3,No log,0.018552


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



ETAPA 5: Avaliação do modelo

Avaliando o modelo em exemplos de teste:

Exemplo 1: Paciente apresenta-se calmo, consciente e orientado, com acesso venoso periférico em membro superior direito, recebendo ceftriaxona 1g EV de 12/12h.
Entidades detectadas:
- acesso (DISPOSITIVO)
- ceftriaxona (MEDICAMENTO)

Exemplo 2: Realizado teste glicêmico com resultado de 145 mg/dl, administrado insulina regular 4 UI conforme prescrição.
Entidades detectadas:
- 145 (RESULTADO_EXAME)
- administrado (PROCEDIMENTO)

Exemplo 3: Paciente em uso de cateter de O2 a 2l/min, apresentando dispneia aos esforços e com monitorização contínua.
Entidades detectadas:
- cateter (DISPOSITIVO)

ETAPA 6: Salvamento do modelo
Modelo salvo em: ./medical-ner-model-final

PIPELINE DE TREINAMENTO CONCLUÍDO COM SUCESSO!


# Demonstração de Uso

#### Texto de exemplo para demonstração

In [ ]:
texto_exemplo = """
PACIENTE COM 24 HORAS DE PÓS-OPERATÓRIO DE LAPAROTOMIA EXPLORADORA,
EVOLUINDO COM DOR MODERADA EM FERIDA OPERATÓRIA.
MEDICADO COM DIPIRONA 2G EV E TRAMADOL 100MG EV CONFORME PRESCRIÇÃO.
MANTÉM ACESSO VENOSO PERIFÉRICO EM MSE, CURATIVO CIRÚRGICO LIMPO E SECO.
SINAIS VITAIS: PA 130/85 MMHG, FC 88 BPM, TAX 36.8°C, SAT 96% EM AR AMBIENTE.
"""

#### Limpeza do texto, detecção das entidades e exibição das entidades detectadas

In [ ]:
texto_limpo = clean_medical_text(texto_exemplo)

entidades = predict_entities(texto_limpo, model, tokenizer)

print("\nEntidades detectadas no texto de exemplo:")
for entidade in entidades:
    print(f"- {entidade['text']} ({entidade['label']})")


Entidades detectadas no texto de exemplo:
- dipirona (MEDICAMENTO)
- acesso (DISPOSITIVO)
- venoso (DISPOSITIVO)
- mse (DISPOSITIVO)
- curativo (PROCEDIMENTO)


#### Visualização do texto com marcações

In [ ]:
texto_marcado = visualize_entities(texto_limpo, entidades)
print("\nTexto com marcações de entidades:")
print(texto_marcado)


Texto com marcações de entidades:
paciente com 24 horas de pos-operatorio de laparotomia exploradora, evoluindo com dor moderada em ferida operatoria. medicado com dipirona 2g [ev](MEDICAMENTO) e tramadol 100mg ev conforme prescricao. mantem acesso venoso periferico [em](DISPOSITIVO) [mse,](DISPOSITIVO) curativo cirurgico [limpo](DISPOSITIVO) e [seco.](PROCEDIMENTO) sinais vitais: pa 130/85 mmhg, fc 88 bpm, tax 36.8°c, sat 96% em ar ambiente.


#Uso em Aplicações Reais
#### `Extract_medical_ner:` Extrai e organiza informações médicas por categoria, retornando um dicionário estruturado.


    Extrai e organiza informações médicas de um texto

    Args:
        text: Texto médico a ser analisado
        model: Modelo NER treinado
        tokenizer: Tokenizer do modelo

    Returns:
        Dicionário com informações médicas organizadas por categoria


In [ ]:
def extract_medical_info(text, model, tokenizer):
    #extrai entidades
    entities = predict_entities(text, model, tokenizer)
    # Organiza por categoria
    info = {
        "MEDICAMENTOS": [],
        "PROCEDIMENTOS": [],
        "DISPOSITIVOS": [],
        "CONDIÇÕES": [],
        "RESULTADOS_EXAMES": []
    }
    # Mapeia as entidades para as categorias
    for entity in entities:
        if entity["label"] == "MEDICAMENTO":
            info["MEDICAMENTOS"].append(entity["text"])
        elif entity["label"] == "PROCEDIMENTO":
            info["PROCEDIMENTOS"].append(entity["text"])
        elif entity["label"] == "DISPOSITIVO":
            info["DISPOSITIVOS"].append(entity["text"])
        elif entity["label"] == "CONDIÇÃO_PACIENTE":
            info["CONDIÇÕES"].append(entity["text"])
        elif entity["label"] == "RESULTADO_EXAME":
            info["RESULTADOS_EXAMES"].append(entity["text"])
    # Remove categorias vazias
    info = {k: v for k, v in info.items() if v}
    return info

#### Exemplo de extração de informações para uso em aplicação

In [ ]:
prontuario = """
PACIENTE INTERNADA HÁ 3 DIAS, APRESENTANDO MELHORA DO QUADRO RESPIRATÓRIO.
MANTÉM O2 SUPLEMENTAR POR CATETER NASAL A 2L/MIN, SAT 95%.
SINAIS VITAIS ESTÁVEIS: PA 120/70MMHG, FC 76BPM, AFEBRIL.
DIURESE PRESENTE, ELIMINAÇÕES PRESENTES, ACEITANDO DIETA VO.
MEDICAÇÕES EM USO: CEFTRIAXONA 1G EV 12/12H, OMEPRAZOL 40MG EV 1X/DIA,
METOCLOPRAMIDA 10MG EV SE NÁUSEAS, DIPIRONA 1G EV 6/6H SE DOR.
EXAMES LABORATORIAIS: HB 11.2, LEUCO 9500, PCR 4.8.
AGUARDANDO ALTA HOSPITALAR.
"""

info_médica = extract_medical_info(prontuario, model, tokenizer)
print("\nInformações médicas extraídas do prontuário:")
for categoria, itens in info_médica.items():
    print(f"\n{categoria}:")
    for item in itens:
        print(f"- {item}")


Informações médicas extraídas do prontuário:

MEDICAMENTOS:
- ceftriaxona
- omeprazol
- dipirona

DISPOSITIVOS:
- o2
- cateter

CONDIÇÕES:
- afebril


#Implementação de Extrações via Prompt


In [ ]:
import requests
import json
import os
from IPython.display import display, Markdown

# Função para criar prompts estruturados para extração de informações médicas
def create_medical_extraction_prompt(text, extraction_type="completo"):
    """
    Cria um prompt específico para extrair informações médicas com base no tipo solicitado

    Args:
        text: Texto médico para análise
        extraction_type: Tipo de extração ("medicamentos", "exames", "diagnosticos" ou "completo")

    Returns:
        Prompt formatado para extração
    """
    # Limpa o texto de entrada
    clean_text = clean_medical_text(text)

    # Base comum do prompt
    base_prompt = f"""Você é um assistente especializado em extrair informações médicas de textos clínicos em português.

Analise o seguinte documento médico:

DOCUMENTO: "{clean_text}"

"""

    # Personaliza o prompt com base no tipo de extração
    if extraction_type == "medicamentos":
        prompt = base_prompt + """
Extraia todos os medicamentos mencionados no texto com suas respectivas informações no formato JSON:

{
  "medicamentos": [
    {
      "nome": "nome do medicamento",
      "dose": "dosagem",
      "via": "via de administração",
      "frequencia": "frequência de administração"
    },
    ...
  ]
}

Inclua apenas medicamentos claramente mencionados no texto. Se alguma informação estiver ausente, use null."""

    elif extraction_type == "exames":
        prompt = base_prompt + """
Extraia todos os exames e seus resultados mencionados no texto no formato JSON:

{
  "exames": [
    {
      "nome": "nome do exame",
      "resultado": "resultado do exame",
      "unidade": "unidade de medida",
      "referencia": "valor de referência (se mencionado)"
    },
    ...
  ]
}

Inclua apenas exames claramente mencionados no texto. Se alguma informação estiver ausente, use null."""

    elif extraction_type == "diagnosticos":
        prompt = base_prompt + """
Extraia todos os diagnósticos, condições clínicas e sintomas mencionados no texto no formato JSON:

{
  "diagnosticos": ["diagnóstico 1", "diagnóstico 2", ...],
  "condicoes": ["condição 1", "condição 2", ...],
  "sintomas": ["sintoma 1", "sintoma 2", ...]
}

Inclua apenas informações claramente mencionadas no texto."""

    else:  # extração completa
        prompt = base_prompt + """
Extraia as seguintes informações do texto no formato JSON:

{
  "medicamentos": [
    {
      "nome": "nome do medicamento",
      "dose": "dosagem",
      "via": "via de administração",
      "frequencia": "frequência"
    },
    ...
  ],
  "dispositivos": ["dispositivo 1", "dispositivo 2", ...],
  "sinais_vitais": [
    {
      "tipo": "tipo do sinal vital",
      "valor": "valor",
      "unidade": "unidade de medida"
    },
    ...
  ],
  "diagnosticos": ["diagnóstico 1", "diagnóstico 2", ...],
  "procedimentos": ["procedimento 1", "procedimento 2", ...]
}

Inclua apenas informações claramente mencionadas no texto. Se alguma categoria não tiver informações, retorne uma lista vazia."""

    return prompt

In [ ]:
os.environ['OPENAI_API_KEY'] = token

In [ ]:
# Função para extrair informações usando serviço de LLM (exemplo com OpenAI)
def extract_via_prompt(text, extraction_type="completo", model="gpt-3.5-turbo"):
    """
    Extrai informações médicas de um texto usando um LLM via API

    Args:
        text: Texto médico para análise
        extraction_type: Tipo de extração
        model: Modelo a ser usado

    Returns:
        Informações estruturadas extraídas do texto
    """
    # Cria o prompt
    prompt = create_medical_extraction_prompt(text, extraction_type)

    try:
        print("Prompt enviado para o modelo:")
        print("-" * 50)
        print(prompt)
        print("-" * 50)

        # Simulação de resposta
        if extraction_type == "medicamentos":
            sample_response = {
                "medicamentos": [
                    {
                        "nome": "CEFTRIAXONA",
                        "dose": "1G",
                        "via": "EV",
                        "frequencia": "12/12H"
                    },
                    {
                        "nome": "DIPIRONA",
                        "dose": "1G",
                        "via": "EV",
                        "frequencia": "6/6H SE DOR"
                    }
                ]
            }
        elif extraction_type == "completo":
            # Simula uma resposta completa
            if "FEBRIL" in text:
                sample_response = {
                    "medicamentos": [
                        {
                            "nome": "CEFTRIAXONA",
                            "dose": "1G",
                            "via": "EV",
                            "frequencia": "12/12H"
                        }
                    ],
                    "dispositivos": ["CATETER O2"],
                    "sinais_vitais": [
                        {
                            "tipo": "TEMPERATURA",
                            "valor": "38.2",
                            "unidade": "°C"
                        },
                        {
                            "tipo": "FC",
                            "valor": "110",
                            "unidade": "BPM"
                        }
                    ],
                    "diagnosticos": ["PNEUMONIA"],
                    "procedimentos": []
                }
            else:
                # Resposta genérica
                sample_response = {
                    "medicamentos": [],
                    "dispositivos": [],
                    "sinais_vitais": [],
                    "diagnosticos": [],
                    "procedimentos": []
                }
        else:
            sample_response = {"resultado": "Tipo de extração não reconhecido"}

        return sample_response

    except Exception as e:
        print(f"Erro ao processar a extração via prompt: {str(e)}")
        return {"erro": str(e)}


In [ ]:
def compare_extraction_methods(text):
    """
    Compara os resultados de extração via NER e via prompt

    Args:
        text: Texto médico para análise

    Returns:
        Comparação dos resultados
    """
    # Extração via NER (usando o modelo treinado anteriormente)
    ner_entities = predict_entities(text, model, tokenizer)

    # Organiza as entidades por categoria
    ner_results = {
        "MEDICAMENTO": [],
        "DISPOSITIVO": [],
        "PROCEDIMENTO": [],
        "CONDIÇÃO_PACIENTE": [],
        "RESULTADO_EXAME": []
    }

    for entity in ner_entities:
        category = entity["label"]
        if category in ner_results:
            ner_results[category].append(entity["text"])

    # Extração via prompt (LLM)
    prompt_results = extract_via_prompt(text, "completo")

    # Exibe os resultados
    print("\n===== COMPARAÇÃO DE MÉTODOS DE EXTRAÇÃO =====\n")

    print("Texto analisado:")
    print("-" * 50)
    print(text)
    print("-" * 50)

    print("\nResultados via NER:")
    for category, items in ner_results.items():
        if items:
            print(f"- {category}: {', '.join(items)}")

    print("\nResultados via Prompt (LLM):")
    for category, items in prompt_results.items():
        if items and len(items) > 0:
            if isinstance(items, list) and isinstance(items[0], dict):
                print(f"- {category}:")
                for item in items:
                    print(f"  * {' | '.join([f'{k}: {v}' for k, v in item.items() if v])}")
            else:
                print(f"- {category}: {', '.join(items)}")

    return {"ner": ner_results, "prompt": prompt_results}

In [ ]:
# Demonstração de uso
def demonstrate_prompt_extraction():
    """
    Demonstra a extração de informações via prompt em exemplos médicos
    """
    examples = [
        """PACIENTE APRESENTA-SE FEBRIL (TAX 38.2°C), TAQUICÁRDICO (FC 110BPM), COM DISPNEIA AOS PEQUENOS ESFORÇOS.
        AUSCULTA PULMONAR COM CREPITAÇÕES EM BASE DE HEMITÓRAX DIREITO. DIAGNOSTICADO COM PNEUMONIA,
        INICIADO TRATAMENTO COM CEFTRIAXONA 1G EV 12/12H E AZITROMICINA 500MG VO 1X/DIA POR 7 DIAS.""",

        """PACIENTE INTERNADO HÁ 3 DIAS, APRESENTANDO MELHORA DO QUADRO RESPIRATÓRIO.
        MANTÉM O2 SUPLEMENTAR POR CATETER NASAL A 2L/MIN, SAT 95%.
        SINAIS VITAIS ESTÁVEIS: PA 120/70MMHG, FC 76BPM, AFEBRIL.
        MEDICAÇÕES EM USO: CEFTRIAXONA 1G EV 12/12H, OMEPRAZOL 40MG EV 1X/DIA,
        METOCLOPRAMIDA 10MG EV SE NÁUSEAS, DIPIRONA 1G EV 6/6H SE DOR."""
    ]

    print("\n===== DEMONSTRAÇÃO DE EXTRAÇÃO VIA PROMPT =====\n")

    for i, example in enumerate(examples):
        print(f"\nExemplo {i+1}:")
        print("-" * 50)
        print(example)
        print("-" * 50)

        print("\nExtração de medicamentos via prompt:")
        meds = extract_via_prompt(example, "medicamentos")
        for med in meds.get("medicamentos", []):
            med_str = f"{med['nome']}"
            if med.get('dose'): med_str += f" {med['dose']}"
            if med.get('via'): med_str += f" {med['via']}"
            if med.get('frequencia'): med_str += f" {med['frequencia']}"
            print(f"- {med_str}")

        print("\nComparação com NER:")
        compare_extraction_methods(example)

        print("\n" + "=" * 50)

In [ ]:
demonstrate_prompt_extraction()


===== DEMONSTRAÇÃO DE EXTRAÇÃO VIA PROMPT =====


Exemplo 1:
--------------------------------------------------
PACIENTE APRESENTA-SE FEBRIL (TAX 38.2°C), TAQUICÁRDICO (FC 110BPM), COM DISPNEIA AOS PEQUENOS ESFORÇOS.
        AUSCULTA PULMONAR COM CREPITAÇÕES EM BASE DE HEMITÓRAX DIREITO. DIAGNOSTICADO COM PNEUMONIA,
        INICIADO TRATAMENTO COM CEFTRIAXONA 1G EV 12/12H E AZITROMICINA 500MG VO 1X/DIA POR 7 DIAS.
--------------------------------------------------

Extração de medicamentos via prompt:
Prompt enviado para o modelo:
--------------------------------------------------
Você é um assistente especializado em extrair informações médicas de textos clínicos em português.

Analise o seguinte documento médico:

DOCUMENTO: "paciente apresenta-se febril (tax 38.2°c), taquicardico (fc 110bpm), com dispneia aos pequenos esforcos.
        ausculta pulmonar com crepitacoes em base de hemitorax direito. diagnosticado com pneumonia,
        iniciado tratamento com ceftriaxona 1g ev 12/12

In [ ]:
# Create a list of libraries to check
libraries = {
    "pandas": pd,
    "nltk": nltk,
    "torch": torch,
    "numpy": np,
    "sklearn": train_test_split.__module__.split('.')[0],
    "transformers": AutoTokenizer.__module__.split('.')[0],
    "datasets": Dataset.__module__.split('.')[0],
    "huggingface_hub": login.__module__.split('.')[0],
    "requests": requests,
    "re": re,
    "json": json,
    "unicodedata": unicodedata
}

# Print the version of each library
for name, lib in libraries.items():
    try:
        version = lib.__version__
        print(f"{name}=={version}")
    except AttributeError:
        try:
            # For packages like re, json, etc. that don't have __version__
            module = __import__(name)
            if hasattr(module, "__version__"):
                print(f"{name}=={module.__version__}")
            else:
                print(f"{name} (version not available)")
        except ImportError:
            print(f"{name} (not installed)")

pandas==2.2.2
nltk==3.9.1
torch==2.6.0+cu124
numpy==1.26.4
sklearn==1.6.1
transformers==4.48.3
datasets==3.4.0
huggingface_hub==0.28.1
requests==2.32.3
re==2.2.1
json==2.0.9
unicodedata (version not available)


In [ ]:
#conferindo versão do python
import sys
python_version = sys.version.split()[0]
print(f"Versão do Python: {python_version}")

Versão do Python: 3.11.11
